# **Economic Dynamics & Social Protection in Ireland**

## Goal
The goal of this project is to understand how labour force participation and social protection policies interact to shape economic health. Using Python for data analysis, the project explore how employment, unemployment, and welfare programs have evolved from 1998 to 2023, and what these patterns reveal about the effectiveness of Ireland’s policies and the resilience of its economy.

## Setup

#### Importing libraries

In [109]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter 
import seaborn as sns
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from scipy import stats
import plotly.io as pio
from IPython.display import Image

%matplotlib inline

## Data Preparation

#### Social Protection Dataset

In [110]:
# Load data
datasp = pd.read_csv('SocialProt.csv')
datasp.head()

,period,programme,scheme,scheme_description,basis,county,county_code,UGI,recipients
0,2014Q1,Children,CB,Child Benefit,Social Assistance,All,All,XX,623739.0
1,2014Q1,Children,FIS,Family Income Supplement,Social Assistance,All,All,XX,48327.0
2,2014Q1,Children,CB,Child Benefit,Social Assistance,Carlow,CW,2ae19629-143d-13a3-e055-000000000001,8319.0
3,2014Q1,Children,FIS,Family Income Supplement,Social Assistance,Carlow,CW,2ae19629-143d-13a3-e055-000000000001,774.0
4,2014Q1,Children,CB,Child Benefit,Social Assistance,Cavan,CN,2ae19629-1448-13a3-e055-000000000001,10349.0


In [111]:
# Filter columns
columns_selection = ['period', 'scheme_description', 'county', 'recipients']
datasp = datasp[columns_selection]
datasp.head()

,period,scheme_description,county,recipients
0,2014Q1,Child Benefit,All,623739.0
1,2014Q1,Family Income Supplement,All,48327.0
2,2014Q1,Child Benefit,Carlow,8319.0
3,2014Q1,Family Income Supplement,Carlow,774.0
4,2014Q1,Child Benefit,Cavan,10349.0


In [112]:
datasp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23894 entries, 0 to 23893
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   period              23894 non-null  object 
 1   scheme_description  23894 non-null  object 
 2   county              23894 non-null  object 
 3   recipients          23894 non-null  float64
dtypes: float64(1), object(3)
memory usage: 746.8+ KB


In [113]:
# Check duplicates
datasp.duplicated().sum() 

0

In [114]:
# Check null values
datasp.isnull().sum()

period                0
scheme_description    0
county                0
recipients            0
dtype: int64

In [115]:
# Rename columns
datasp.rename(columns= {'period': 'Year',
                        'county': 'Residence_county',
                        'scheme_description': 'Scheme_description',
                        'recipients': 'Recipients'}, inplace=True)

In [116]:
# Count residence_county values
datasp['Residence_county'].value_counts()

Residence_county
All          874
Limerick     853
Wexford      853
Westmeath    853
Waterford    853
Tipperary    853
Offaly       853
Monaghan     853
Meath        853
Mayo         853
Louth        853
Carlow       853
Longford     853
Leitrim      853
Laois        853
Kilkenny     853
Kildare      853
Kerry        853
Galway       853
Dublin       853
Donegal      853
Cork         853
Clare        853
Wicklow      853
Roscommon    852
Sligo        852
Cavan        852
Unknown      845
Name: count, dtype: int64

In [117]:
# Remove Unknown data and all values from residence_county column
datasp = datasp[datasp['Residence_county'] != 'Unknown']
datasp = datasp[datasp['Residence_county'] != 'All']

In [118]:
# Change recipients type to integer
datasp['Recipients'] = datasp['Recipients'].astype(int)
datasp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22175 entries, 2 to 23893
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Year                22175 non-null  object
 1   Scheme_description  22175 non-null  object
 2   Residence_county    22175 non-null  object
 3   Recipients          22175 non-null  int32 
dtypes: int32(1), object(3)
memory usage: 779.6+ KB


In [119]:
# Split year in two columns (year and period)
datasp[['Year', 'Period']] = datasp['Year'].str.split('Q', expand=True) 

In [120]:
# Convert year and periods to integer
datasp['Period'] = datasp['Period'].astype(int)
datasp['Year'] = datasp['Year'].astype(int)

In [121]:
# Organize columns and reset index
columns_selection = ['Year', 'Period', 'Scheme_description', 'Residence_county', 'Recipients']
datasp = datasp[columns_selection]
datasp.reset_index(drop=True, inplace=True)
datasp.head()

,Year,Period,Scheme_description,Residence_county,Recipients
0,2014,1,Child Benefit,Carlow,8319
1,2014,1,Family Income Supplement,Carlow,774
2,2014,1,Child Benefit,Cavan,10349
3,2014,1,Family Income Supplement,Cavan,1089
4,2014,1,Child Benefit,Clare,15958


In [122]:
datasp.shape

(22175, 5)

#### Labour Force Dataset

In [123]:
# Load dataset
labour_df = pd.read_csv("Labour_force .csv")
labour_df.head()

,STATISTIC,Statistic Label,TLIST(Q1),Quarter,C02199V02655,Sex,C01999V02434,ILO Economic Status,UNIT,VALUE
0,QLF01C01,Persons aged 15 years and over,19981,1998Q1,-,Both sexes,-,All ILO economic status,Thousand,2857.2
1,QLF01C01,Persons aged 15 years and over,19981,1998Q1,-,Both sexes,01,In labour force,Thousand,1699.0
2,QLF01C01,Persons aged 15 years and over,19981,1998Q1,-,Both sexes,02,In employment,Thousand,1550.3
3,QLF01C01,Persons aged 15 years and over,19981,1998Q1,-,Both sexes,03,In employment full-time,Thousand,1282.6
4,QLF01C01,Persons aged 15 years and over,19981,1998Q1,-,Both sexes,04,In employment part-time,Thousand,267.7


In [124]:
# Filter Sex column to both sexes
labour_df = labour_df[labour_df["Sex"] == "Both sexes"]

In [125]:
# Filter Columns
columns_labour = ["Quarter", "ILO Economic Status", "VALUE"]
labour_df = labour_df[columns_labour]

In [126]:
# Check variables in economic status
labour_df["ILO Economic Status"].unique()

array(['All ILO economic status', 'In labour force', 'In employment',
       'In employment full-time', 'In employment part-time',
       'In employment part-time - not underemployed',
       'In employment part-time - underemployed', 'Unemployed',
       'Unemployed seeking full-time work/future job-starter',
       'Unemployed seeking part-time work', 'Not in labour force',
       'Potential additional labour force', 'Others not in labour force',
       'Unemployed seeking work as self-employed'], dtype=object)

In [127]:
# Filter ILO Economic Status column: In labour force / In employment / Unemployed / Not in labour force
labour_df = labour_df[labour_df["ILO Economic Status"].isin(["In labour force", "In employment", "Unemployed", "Not in labour force"])].reset_index(drop=True)
labour_df.head()

,Quarter,ILO Economic Status,VALUE
0,1998Q1,In labour force,1699.0
1,1998Q1,In employment,1550.3
2,1998Q1,Unemployed,148.7
3,1998Q1,Not in labour force,1158.2
4,1998Q2,In labour force,1710.7


In [128]:
# Check null
labour_df.isnull().sum()

Quarter                0
ILO Economic Status    0
VALUE                  0
dtype: int64

In [129]:
# Check duplicates
labour_df.duplicated().sum()

0

In [130]:
# Split year in two columns (year and period)
labour_df[["year", "period"]] = labour_df["Quarter"].str.split("Q", expand=True)

In [131]:
# Rename columns
labour_df.rename(columns={
                          "ILO Economic Status": "economic_status",
                          "VALUE": "individuals"
                          }, inplace=True)

In [132]:
# Organize columns
columns_order = ["year", "period", "economic_status", "individuals"]
labour_df = labour_df[columns_order]

# Set the individuals columns to thousands
labour_df["individuals"] = labour_df["individuals"]*1000

In [133]:
# Check outliers
Q1 = labour_df['individuals'].quantile(.25)
Q3 = labour_df['individuals'].quantile(.75)
IQR = Q3 - Q1
lower_out = Q1 - 1.5 * IQR
upper_out = Q3 + 1.5 * IQR

outliers = labour_df[(labour_df['individuals'] < lower_out) | (labour_df['individuals'] > upper_out)]
outliers.sum()


year                 0
period               0
economic_status      0
individuals        0.0
dtype: object

In [134]:
# Change data type to integer
labour_df["year"] = labour_df["year"].astype(np.int64)
labour_df["period"] = labour_df["period"].astype(np.int64)

In [135]:
labour_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 428 entries, 0 to 427
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   year             428 non-null    int64  
 1   period           428 non-null    int64  
 2   economic_status  428 non-null    object 
 3   individuals      428 non-null    float64
dtypes: float64(1), int64(2), object(1)
memory usage: 13.5+ KB


In [136]:
labour_df.head()

,year,period,economic_status,individuals
0,1998,1,In labour force,1699000.0
1,1998,1,In employment,1550300.0
2,1998,1,Unemployed,148700.0
3,1998,1,Not in labour force,1158200.0
4,1998,2,In labour force,1710700.0


#### Social Protection Expenditure Dataset

In [137]:
# Load dataset
expenditure_df = pd.read_csv("Expenditure.csv")
expenditure_df.head()

,STATISTIC,Statistic Label,TLIST(A1),Year,C03908V04660,Social Protection Expenditure,UNIT,VALUE
0,SPEA02C01,Social Protection Expenditure by function,2000,2000,1000000,Total Social Protection Expenditure,€million,16466
1,SPEA02C01,Social Protection Expenditure by function,2000,2000,1100000,"Expenditure, Social Protection Benefits",€million,15584
2,SPEA02C01,Social Protection Expenditure by function,2000,2000,1110000,"Expenditure, Sickness benefits",€million,5517
3,SPEA02C01,Social Protection Expenditure by function,2000,2000,1120000,"Expenditure, Disability benefits",€million,682
4,SPEA02C01,Social Protection Expenditure by function,2000,2000,1130000,"Expenditure, Old age benefits",€million,4444


In [138]:
# Remove columns
expenditure_df.drop(columns=["STATISTIC", "Statistic Label", "TLIST(A1)", "C03908V04660"], inplace=True)

In [139]:
# Check null
expenditure_df.isnull().sum()

Year                             0
Social Protection Expenditure    0
UNIT                             0
VALUE                            0
dtype: int64

In [140]:
# Check duplicates
expenditure_df.duplicated().sum()

0

In [141]:
# Rename columns
expenditure_df.rename(columns={"Year": "year",
                       "Social Protection Expenditure": "social protection expenditure",
                       "VALUE": "value (euro)",
                       "UNIT": "unit"}, inplace=True)

In [142]:
# Rename variables in social protection expenditure column for readability 
social_expenditure_rename = {
                    "Total Social Protection Expenditure": "Total Social Protection",
                    "Expenditure, Social Protection Benefits": "Social Protection Benefits", 
                    "Expenditure, Sickness benefits": "Sickness benefits",
                    "Expenditure, Disability benefits": "Disability benefits",
                    "Expenditure, Old age benefits": "Old age benefits",
                    "Expenditure, Survivor benefits": "Survivor benefits",
                    "Expenditure, Family benefits": "Family benefits",
                    "Expenditure, Unemployment benefits": "Unemployment benefits",
                    "Expenditure, Housing benefits": "Housing benefits",
                    "Expenditure, Social exclusion benefits n.e.c.": "Social exclusion benefits",
                    "Expenditure, Social Protection - Administration Costs": "Social Protection - Administration Costs" 
}

expenditure_df["social protection expenditure"] = expenditure_df["social protection expenditure"].replace(social_expenditure_rename)

In [143]:
# Rename variables in unit column
unit_rename = {"€million": "Million"}
expenditure_df["unit"] = expenditure_df["unit"].replace(unit_rename)

In [144]:
# Check unique values 
expenditure_df["social protection expenditure"].unique()

array(['Total Social Protection', 'Social Protection Benefits',
       'Sickness benefits', 'Disability benefits', 'Old age benefits',
       'Survivor benefits', 'Family benefits', 'Unemployment benefits',
       'Housing benefits', 'Social exclusion benefits',
       'Social Protection - Administration Costs'], dtype=object)

In [145]:
# Check outliers
Q1 = expenditure_df['value (euro)'].quantile(.25)
Q3 = expenditure_df['value (euro)'].quantile(.75)
IQR = Q3 - Q1
lower_out = Q1 - 1.5 * IQR
upper_out = Q3 + 1.5 * IQR

outliers = expenditure_df[(expenditure_df['value (euro)'] < lower_out) | (expenditure_df['value (euro)'] > upper_out)]
outliers.head()

,year,social protection expenditure,unit,value (euro)
77,2007,Total Social Protection,Million,35682
78,2007,Social Protection Benefits,Million,34080
88,2008,Total Social Protection,Million,39209
89,2008,Social Protection Benefits,Million,37838
99,2009,Total Social Protection,Million,42094


Not necessary to remove the outliers since the main analysis will be made on unemployment social protection.

In [146]:
expenditure_df.head()

,year,social protection expenditure,unit,value (euro)
0,2000,Total Social Protection,Million,16466
1,2000,Social Protection Benefits,Million,15584
2,2000,Sickness benefits,Million,5517
3,2000,Disability benefits,Million,682
4,2000,Old age benefits,Million,4444
